In [26]:
# Zelle 1: Importieren der benötigten Bibliotheken
import json
import os
import glob
from pathlib import Path

In [27]:
# Zelle 2: Definition der Eingabe- und Ausgabepfade
# Annahme: Die JSON-Dateien befinden sich im Unterordner "input_files"
input_dir = "modified_files_json"
output_dir = "new_versions_json"

# Erstelle den Ausgabeordner, falls er nicht existiert
os.makedirs(output_dir, exist_ok=True)

# Liste aller JSON-Dateien im Eingabeordner
json_files = glob.glob(os.path.join(input_dir, "*.json"))
print(f"Gefundene JSON-Dateien: {len(json_files)}")
for file in json_files:
    print(f"- {os.path.basename(file)}")


Gefundene JSON-Dateien: 5
- clothingFinal.json
- crowdFinal.json
- environmentFinal.json
- graphicalrepresentationsFinal.json
- interactionFinal.json


In [28]:
# Zelle 3: Hilfsfunktion zum Laden und Überprüfen der JSON-Struktur
def load_and_explore_json(file_path):
    """Lädt eine JSON-Datei und analysiert ihre Struktur"""
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Projektnamen identifizieren
    projects = set()
    if isinstance(data, list):  # Falls die Datei eine Liste von Tasks enthält
        for task in data:
            for annotation in task.get('annotations', []):
                for result in annotation.get('result', []):
                    if 'from_name' in result:
                        projects.add(result['from_name'])
    else:  # Falls die Datei einen einzelnen Task enthält
        for annotation in data.get('annotations', []):
            for result in annotation.get('result', []):
                if 'from_name' in result:
                    projects.add(result['from_name'])
    
    return data, projects

# Testen mit der ersten Datei
if json_files:
    sample_data, projects = load_and_explore_json(json_files[0])
    print(f"Gefundene Projekte in der ersten Datei: {projects}")


Gefundene Projekte in der ersten Datei: {'keine_merkmale', 'Clothing'}


In [29]:
# Zelle 4: Hauptfunktion zum Extrahieren der Annotationen aus allen Dateien
def extract_annotations_from_files(json_files):
    """Extrahiert Annotationen aus allen Datei und gibt ein Dictionary mit Bild-zu-Labels-Zuordnung zurück"""
    all_annotations = {}
    
    for file_path in json_files:
        data, _ = load_and_explore_json(file_path)
        
        # Verarbeiten als Liste oder als einzelnes Objekt
        tasks = data if isinstance(data, list) else [data]
        
        for task in tasks:
            # Bildpfad extrahieren
            image_path = task.get('data', {}).get('image', '')
            if not image_path:
                continue
                
            # Nur den Dateinamen verwenden
            image_name = os.path.basename(image_path)
            #print("image_name vor der Kürzung: "+image_name)

            # Doppelpunkte im image_name durch Unterstriche ersetzen
            image_name = image_name.replace(":", "_")
            #print("image_name nach der Kürzung: "+image_name)
            
            
            
            # Initialisieren der Annotation für dieses Bild, falls noch nicht vorhanden
            if image_name not in all_annotations:
                all_annotations[image_name] = {
                    'Graphical Representations': [],
                    'Clothing': [],
                    'Crowd': [],
                    'Interaction': [],
                    'Environment': []
                }
            
            # Alle Annotationen für dieses Bild durchgehen
            for annotation in task.get('annotations', []):
                for result in annotation.get('result', []):
                    # Projektname und Labels extrahieren
                    project_name = result.get('from_name', '')
                    choices = result.get('value', {}).get('choices', [])
                    
                    # "no_features" überspringen
                    choices = [c for c in choices if c != "no_features"]
                    
                    # Labels zum entsprechenden Projekt hinzufügen
                    if project_name in all_annotations[image_name]:
                        all_annotations[image_name][project_name].extend(choices)
    
    # Duplikate entfernen
    for image_name, projects in all_annotations.items():
        for project_name, labels in projects.items():
            all_annotations[image_name][project_name] = list(set(labels))
    
    return all_annotations

# Extrahiere alle Annotationen
all_image_annotations = extract_annotations_from_files(json_files)
print(f"Anzahl der gefundenen Bilder: {len(all_image_annotations)}")


Anzahl der gefundenen Bilder: 308


In [30]:
print(all_image_annotations)


{'olafscholz_26-09-2021_00_00_video7.jpeg': {'Graphical Representations': ['campaignParaphernalia'], 'Clothing': ['informalAttire'], 'Crowd': ['largeAudiences'], 'Interaction': [], 'Environment': ['identifiableEntourage']}, 'christlichsozialeunion_18-09-2021_00_00_image1.jpeg': {'Graphical Representations': ['campaignParaphernalia', 'patrioticSymbols'], 'Clothing': ['formalAttire'], 'Crowd': [], 'Interaction': [], 'Environment': ['identifiableEntourage']}, '2021-09-24_11-50-01_UTC_3.jpg': {'Graphical Representations': [], 'Clothing': ['informalAttire'], 'Crowd': ['largeAudiences'], 'Interaction': ['children'], 'Environment': ['ordinaryPeople']}, 'christianlindner_25-09-2021_00_00_video0.jpeg': {'Graphical Representations': [], 'Clothing': ['informalAttire'], 'Crowd': [], 'Interaction': ['interactionWithSupporters'], 'Environment': ['ordinaryPeople']}, '2021-09-13_19-36-55_UTC_3.jpg': {'Graphical Representations': ['campaignParaphernalia'], 'Clothing': ['informalAttire'], 'Crowd': [], '

In [31]:
# Zelle 5: Erstellen und Speichern der ersten JSON-Datei (Alle Labels)
def create_first_json(annotations):
    """Erstellt das erste JSON-Format mit allen Labels"""
    result = {}
    
    for image_name, projects in annotations.items():
        # Alle Labels aus allen Projekten zusammenführen
        all_labels = []
        for project_name, labels in projects.items():
            all_labels.extend(labels)
        
        # Duplikate entfernen
        all_labels = list(set(all_labels))
        
        # Zum Ergebnis hinzufügen
        result[image_name] = all_labels
    
    return result

# Erstellen und Speichern der ersten JSON-Datei
first_json = create_first_json(all_image_annotations)
first_json_path = os.path.join(output_dir, "all_labels.json")
with open(first_json_path, 'w', encoding='utf-8') as f:
    json.dump(first_json, f, indent=2)

print(f"Erste JSON-Datei gespeichert: {first_json_path}")


Erste JSON-Datei gespeichert: new_versions_json\all_labels.json


In [32]:
# Zelle 6: Definition der übergeordneten Kategorien
category_mapping = {
    "Statesmanship": [
        "campaignParaphernalia",
        "patrioticSymbols",
        "formalAttire",
        "identifiableEntourage",
    ],
    "Compassion": [
        "interactionWithSupporters",
        "physicalEmbraces",
        "affinityGestures",
        "children",
        "admiringPeople",
    ],
    "MassAppeal": ["largeAudiences", "approvingAudiences", "interactionWithCrowds"],
    "Ordinariness": ["informalAttire", "ordinaryPeople"],
}

# Umgekehrtes Mapping für einfacheren Zugriff
label_to_category = {}
for category, labels in category_mapping.items():
    for label in labels:
        label_to_category[label] = category


In [33]:
# Zelle 7: Erstellen und Speichern der zweiten JSON-Datei (Kategorien)
def create_category_json(annotations, label_to_category):
    """Erstellt das zweite JSON-Format mit den übergeordneten Kategorien"""
    result = {}

    for image_name, projects in annotations.items():
        # Alle Labels aus allen Projekten zusammenführen
        all_labels = []
        for project_name, labels in projects.items():
            all_labels.extend(labels)

        # Auf Kategorien mappen
        categories = set()
        for label in all_labels:
            if label in label_to_category:
                categories.add(label_to_category[label])

        # Zum Ergebnis hinzufügen
        result[image_name] = list(categories)

    return result


# Erstellen und Speichern der zweiten JSON-Datei
category_json = create_category_json(all_image_annotations, label_to_category)
category_json_path = os.path.join(output_dir, "category_labels.json")
with open(category_json_path, "w", encoding="utf-8") as f:
    json.dump(category_json, f, indent=2)

print(f"Zweite JSON-Datei gespeichert: {category_json_path}")


Zweite JSON-Datei gespeichert: new_versions_json\category_labels.json


In [34]:
# Zelle 8: Erstellen und Speichern der dritten JSON-Datei (Ausschluss bestimmter Labels)
def create_excluded_json(annotations, excluded_labels):
    """Erstellt das dritte JSON-Format mit Ausschluss bestimmter Labels"""
    result = {}

    for image_name, projects in annotations.items():
        # Alle Labels aus allen Projekten zusammenführen
        all_labels = []
        for project_name, labels in projects.items():
            all_labels.extend(labels)

        # Ausgeschlossene Labels entfernen
        filtered_labels = [
            label for label in all_labels if label not in excluded_labels
        ]

        # Zum Ergebnis hinzufügen
        result[image_name] = filtered_labels

    return result


# Definition der auszuschließenden Labels für die dritte JSON
excluded_labels_1 = ["approvingAudiences", "admiringPeople", "no_features"]

# Erstellen und Speichern der dritten JSON-Datei
excluded_json_1 = create_excluded_json(all_image_annotations, excluded_labels_1)
excluded_json_1_path = os.path.join(output_dir, "excluded_labels_1.json")
with open(excluded_json_1_path, "w", encoding="utf-8") as f:
    json.dump(excluded_json_1, f, indent=2)

print(f"Dritte JSON-Datei gespeichert: {excluded_json_1_path}")


Dritte JSON-Datei gespeichert: new_versions_json\excluded_labels_1.json


In [35]:
# Zelle 9: Erstellen und Speichern der vierten JSON-Datei (Erweiterter Ausschluss)
# Definition der auszuschließenden Labels für die vierte JSON
excluded_labels_2 = [
    "approvingAudiences",
    "admiringPeople",
    "no_features",
    "interactionWithCrowds",  # Korrigiert von "interactionWithCrowd"
    "ordinaryPeople",
    "affinityGestures",
]

# Erstellen und Speichern der vierten JSON-Datei
excluded_json_2 = create_excluded_json(all_image_annotations, excluded_labels_2)
excluded_json_2_path = os.path.join(output_dir, "excluded_labels_2.json")
with open(excluded_json_2_path, "w", encoding="utf-8") as f:
    json.dump(excluded_json_2, f, indent=2)

print(f"Vierte JSON-Datei gespeichert: {excluded_json_2_path}")


Vierte JSON-Datei gespeichert: new_versions_json\excluded_labels_2.json


In [36]:
# Zelle 10: Überprüfung und Zusammenfassung der erstellten Dateien
def summarize_json_file(file_path):
    """Gibt eine Zusammenfassung einer JSON-Datei zurück"""
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    num_images = len(data)
    sample_image = next(iter(data))
    sample_labels = data[sample_image]

    return {
        "Anzahl der Bilder": num_images,
        "Beispiel-Bild": sample_image,
        "Beispiel-Labels": sample_labels,
    }


# Zusammenfassung aller erstellten Dateien ausgeben
output_files = [
    ("Alle Labels", first_json_path),
    ("Kategorien", category_json_path),
    ("Ausgeschlossene Labels 1", excluded_json_1_path),
    ("Ausgeschlossene Labels 2", excluded_json_2_path),
]

for name, path in output_files:
    summary = summarize_json_file(path)
    print(f"\n{name}:")
    print(f"  Anzahl der Bilder: {summary['Anzahl der Bilder']}")
    print(f"  Beispiel-Bild: {summary['Beispiel-Bild']}")
    print(f"  Beispiel-Labels: {summary['Beispiel-Labels']}")



Alle Labels:
  Anzahl der Bilder: 308
  Beispiel-Bild: olafscholz_26-09-2021_00_00_video7.jpeg
  Beispiel-Labels: ['campaignParaphernalia', 'largeAudiences', 'informalAttire', 'identifiableEntourage']

Kategorien:
  Anzahl der Bilder: 308
  Beispiel-Bild: olafscholz_26-09-2021_00_00_video7.jpeg
  Beispiel-Labels: ['Ordinariness', 'Statesmanship', 'MassAppeal']

Ausgeschlossene Labels 1:
  Anzahl der Bilder: 308
  Beispiel-Bild: olafscholz_26-09-2021_00_00_video7.jpeg
  Beispiel-Labels: ['campaignParaphernalia', 'informalAttire', 'largeAudiences', 'identifiableEntourage']

Ausgeschlossene Labels 2:
  Anzahl der Bilder: 308
  Beispiel-Bild: olafscholz_26-09-2021_00_00_video7.jpeg
  Beispiel-Labels: ['campaignParaphernalia', 'informalAttire', 'largeAudiences', 'identifiableEntourage']
